### Playground

This notebook shows how to communicate with configuration registers, status registers and AXI4-Stream interfaces inside the FPGA using the [pyhubio](https://github.com/pavel-demin/pyhubio) library.

Import the required libraries and instantiate `PyhubTCP`.

In [ ]:
from pyhubio import PyhubTCP
import numpy as np
import pylab as pl
import time

io = PyhubTCP("192.168.1.100")

Start I/O and program the FPGA.

In [ ]:
io.start()
io.program("playground.bit")

The port number (hub address) of the configuration registers is 0.

The 8-bit configuration register at address 4 is connected to the LEDs on the Red Pitaya board.

The following script turns all the LEDs on and off several times.

In [ ]:
pins = np.zeros(1, np.uint32)

for i in range(10):
    pins[0] ^= 255
    io.write(pins, port=0, addr=4)
    time.sleep(0.5)

The port number (hub address) of the status registers is 1.

The 32-bit configuration register at address 8 is connected to the 16-bit inputs of the DSP48 multiplier and the status register at address 4 is connected to the 32-bit output of the multiplier.

The following script writes two 16-bit integer values to be multiplied in the configuration register at address 4 and reads the result of the multiplication from the status register at address 4.

In [ ]:
input = np.array([6, 7], np.int16)
io.write(input, port=0, addr=8)

output = np.zeros(1, np.int32)
io.read(output, port=1, addr=4)

print(output[0])

The second AXI4-Stream interface has port number (hub address) 3. Its slave side is connected to a counter that increments after each read transaction.

The following script reads 10 numbers from the counter.

In [ ]:
output = np.zeros(10, np.int32)
io.read(output, port=3, addr=0)

print(output)

The third BRAM interface has port number (hub address) 4. It is connected to a BRAM buffer used by the XADC interface.

The following script reads the slow analog inputs using this BRAM interface.

In [ ]:
buffer = np.zeros(32, np.uint32)
io.read(buffer, port=4, addr=0)

print(buffer[16], buffer[17], buffer[24], buffer[25])

The first BRAM interface has port number (hub address) 2. It is connected to a 64 kB BRAM buffer and the content of this buffer is continuously sent to the DAC interface.

The following scripts show how to generate Gaussian shaped pulses.

Create custom pulse shape.

In [ ]:
def gauss(x, A, x0, sigma):
    return A * np.exp(-((x - x0) ** 2) / (2 * sigma**2))

pulse = gauss(np.arange(0, 16384, 1), 1, 5000, 500)
pulse = np.int16(np.floor(pulse * 8191 + 0.5))

pl.figure(figsize=[8, 2], dpi=150, constrained_layout=True)

pl.plot(pulse)
pl.grid()

Send the pulse shape to the BRAM buffer.

In [ ]:
size = 16384
buffer = np.zeros(size * 2, np.int16)
buffer[0::2] = pulse

io.write(buffer, port=2, addr=0)

The ADC interface is connected to the slave side of the first AXI4-Stream interface via an intermediate 64 kB FIFO buffer.

The following scripts shows how to read ADC samples.

Initialize variables.

In [ ]:
size = 16384
buffer = np.zeros(size * 2, np.int16)
status = np.zeros(1, np.uint16)

Reset the FIFO buffer, wait until the FIFO buffer is full, then read ADC samples from the FIFO buffer and plot the ADC samples separately for two channels.

In [ ]:
io.edge(0, 1, positive=True, addr=0)

while status[0] < size:
    time.sleep(0.1)
    io.read(status, port=1, addr=0)

io.read(buffer, port=2, addr=0)

ch1 = buffer[0::2]
ch2 = buffer[1::2]

pl.figure(figsize=[8, 4], dpi=150, constrained_layout=True)

pl.plot(ch1)
pl.plot(ch2)

pl.xlabel("sample number")
pl.ylabel("ADC units")

pl.ylim(-9000, 9000)
pl.grid()

Stop I/O.

In [ ]:
io.stop()